In [16]:
import pandas as pd
import scipy.stats as sps
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [17]:
oot = pd.read_csv("oot_scaled_final.csv", index_col=0)
data = pd.read_csv("train_test_final.csv", index_col=0)

In [18]:
oot

,card_ps_3d_total,card-state_ps_3d_total,card-state_ps_14d_total,card-state_ps_3d_maximum,card-state_ps_1d_total,card-state_ps_7d_maximum,card-state_ps_30d_total,card_ps_0d_total,card-state_ps_1d_maximum,card-merchant_ps_3d_total,...,card-state_ps_14d_average,card-merchant_ps_1d_total,card_ps_30d_average,card-state_ps_7d_median,merch#_ps_3d_total,card-state_ps_0d_maximum,card-state_ps_14d_median,card-state_ps_0d_total,card-state_ps_0d_average,fraud
0,0.080367,0.000000,0.126722,0.000000,0.000000,0.000000,0.070056,0.000000,0.000000,0.000000,...,0.946919,0.0,-0.031004,0.000000,-0.018568,0.0,0.986825,0.0,0.0,0
1,-0.025350,0.000000,0.044776,0.000000,0.000000,0.000000,0.004337,0.000000,0.000000,0.000000,...,0.386291,0.0,-0.141082,0.000000,-0.018568,0.0,0.404762,0.0,0.0,0
2,0.008234,0.062919,0.081669,0.251368,0.000000,0.222432,0.149647,0.000000,0.000000,0.000000,...,0.206129,0.0,-0.106484,0.409187,-0.018568,0.0,0.150208,0.0,0.0,0
3,-0.025350,0.000000,0.123971,0.000000,0.000000,0.000000,0.075795,0.000000,0.000000,0.000000,...,0.458973,0.0,-0.380419,0.000000,-0.018568,0.0,0.480223,0.0,0.0,0
4,-0.025350,0.000000,-0.013172,0.000000,0.000000,0.000000,0.000826,0.000000,0.000000,0.000000,...,-0.010154,0.0,0.155685,0.000000,-0.018568,0.0,-0.006842,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12422,-0.025350,0.000000,-0.013172,0.000000,0.000000,0.000000,-0.042135,0.000000,0.000000,0.000000,...,-0.010154,0.0,-0.602674,0.000000,-0.018568,0.0,-0.006842,0.0,0.0,0
12423,0.465089,0.233400,0.695686,0.932454,0.000000,0.825116,0.526346,0.221829,0.000000,0.000000,...,1.606376,0.0,0.756168,1.517888,-0.018568,0.0,1.397185,0.0,0.0,0
12424,0.189940,0.021850,0.005332,0.087292,0.022598,0.077244,-0.027295,0.000000,0.092119,0.000000,...,0.116444,0.0,0.317933,0.142098,1.087346,0.0,0.124597,0.0,0.0,0
12425,-0.025350,0.000000,-0.013172,0.000000,0.000000,0.000000,0.070142,0.000000,0.000000,0.000000,...,-0.010154,0.0,0.487731,0.000000,0.699855,0.0,-0.006842,0.0,0.0,0


In [19]:
from sklearn.model_selection import KFold

In [20]:
from sklearn.neural_network import MLPClassifier

In [21]:
x_oot=oot.iloc[:,:30].values
y_oot=oot.iloc[:,30].values

In [22]:
x_oot

array([[ 0.08036692,  0.        ,  0.12672187, ...,  0.9868247 ,
         0.        ,  0.        ],
       [-0.02534977,  0.        ,  0.04477574, ...,  0.40476204,
         0.        ,  0.        ],
       [ 0.0082335 ,  0.06291917,  0.08166927, ...,  0.15020795,
         0.        ,  0.        ],
       ...,
       [ 0.18994024,  0.02184989,  0.0053325 , ...,  0.12459705,
         0.        ,  0.        ],
       [-0.02534977,  0.        , -0.01317221, ..., -0.00684177,
         0.        ,  0.        ],
       [ 0.93278295,  1.00648782,  0.83922415, ...,  0.15845832,
         0.        ,  0.        ]])

In [23]:
X=data.iloc[:,:30].values
y=data.iloc[:,30].values

In [24]:
for n in range(1,21,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0
        fdr_oot = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
            
            NN_oot_pred = NN.predict_proba(x_oot)[:,1]
            ind = np.argsort(NN_oot_pred)
            NN_y_oot_sorted = np.take_along_axis(y_oot, ind, axis=0)
            NN_oot_pred_sorted = np.take_along_axis(NN_oot_pred, ind, axis=0)
            NN_y_oot_sorted_3per = NN_y_oot_sorted[-int(len(NN_y_oot_sorted)*0.03):-1]
            NN_y_oot_sorted_3per_fraud = NN_y_oot_sorted_3per[NN_y_oot_sorted_3per==1]
            y_oot_fraud = len(y_oot[y_oot==1])
            fdr_oot += len(NN_y_oot_sorted_3per_fraud)/y_oot_fraud

        print(f'training: with {n} nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with {n} nodes and {m} alpha, FDR at 3%:', fdr_test/10)
        print(f'testing: with {n} nodes and {m} alpha, FDR at 3%:', fdr_oot/10)

kfold-------
training: with 1 nodes and 0.01 alpha, FDR at 3%: 0.5187610661470923
testing: with 1 nodes and 0.01 alpha, FDR at 3%: 0.5040016782749506
testing: with 1 nodes and 0.01 alpha, FDR at 3%: 0.31396648044692743
kfold-------
training: with 1 nodes and 0.02 alpha, FDR at 3%: 0.5804519449150265
testing: with 1 nodes and 0.02 alpha, FDR at 3%: 0.5654865206225204
testing: with 1 nodes and 0.02 alpha, FDR at 3%: 0.34134078212290503
kfold-------
training: with 1 nodes and 0.04 alpha, FDR at 3%: 0.39890197546598644
testing: with 1 nodes and 0.04 alpha, FDR at 3%: 0.3916626846341893
testing: with 1 nodes and 0.04 alpha, FDR at 3%: 0.24078212290502793
kfold-------
training: with 1 nodes and 0.06 alpha, FDR at 3%: 0.5860394394358501
testing: with 1 nodes and 0.06 alpha, FDR at 3%: 0.5841351684051738
testing: with 1 nodes and 0.06 alpha, FDR at 3%: 0.359217877094972
kfold-------
training: with 1 nodes and 0.08 alpha, FDR at 3%: 0.5197020511407349
testing: with 1 nodes and 0.08 alpha, FDR a

training: with 7 nodes and 0.04 alpha, FDR at 3%: 0.6849774422687706
testing: with 7 nodes and 0.04 alpha, FDR at 3%: 0.665548847525059
testing: with 7 nodes and 0.04 alpha, FDR at 3%: 0.4754189944134078
kfold-------
training: with 7 nodes and 0.06 alpha, FDR at 3%: 0.6696068902782563
testing: with 7 nodes and 0.06 alpha, FDR at 3%: 0.6446687241465134
testing: with 7 nodes and 0.06 alpha, FDR at 3%: 0.44413407821229056
kfold-------
training: with 7 nodes and 0.08 alpha, FDR at 3%: 0.6692549803100116
testing: with 7 nodes and 0.08 alpha, FDR at 3%: 0.6464451596302407
testing: with 7 nodes and 0.08 alpha, FDR at 3%: 0.44134078212290506
kfold-------
training: with 7 nodes and 0.1 alpha, FDR at 3%: 0.6661067644800617
testing: with 7 nodes and 0.1 alpha, FDR at 3%: 0.6433536646415583
testing: with 7 nodes and 0.1 alpha, FDR at 3%: 0.394413407821229
kfold-------
training: with 8 nodes and 0.01 alpha, FDR at 3%: 0.7180858703922628
testing: with 8 nodes and 0.01 alpha, FDR at 3%: 0.69655876550

training: with 13 nodes and 0.08 alpha, FDR at 3%: 0.6697072348545146
testing: with 13 nodes and 0.08 alpha, FDR at 3%: 0.6519729143057785
testing: with 13 nodes and 0.08 alpha, FDR at 3%: 0.44134078212290506
kfold-------
training: with 13 nodes and 0.1 alpha, FDR at 3%: 0.6676928429506531
testing: with 13 nodes and 0.1 alpha, FDR at 3%: 0.6553628824195702
testing: with 13 nodes and 0.1 alpha, FDR at 3%: 0.44301675977653626
kfold-------
training: with 14 nodes and 0.01 alpha, FDR at 3%: 0.7562882313911607
testing: with 14 nodes and 0.01 alpha, FDR at 3%: 0.7229891808738259
testing: with 14 nodes and 0.01 alpha, FDR at 3%: 0.5279329608938547
kfold-------
training: with 14 nodes and 0.02 alpha, FDR at 3%: 0.7167918679694996
testing: with 14 nodes and 0.02 alpha, FDR at 3%: 0.6929030309299242
testing: with 14 nodes and 0.02 alpha, FDR at 3%: 0.5273743016759777
kfold-------
training: with 14 nodes and 0.04 alpha, FDR at 3%: 0.6970279000980938
testing: with 14 nodes and 0.04 alpha, FDR at 3

training: with 20 nodes and 0.01 alpha, FDR at 3%: 0.7619156203338856
testing: with 20 nodes and 0.01 alpha, FDR at 3%: 0.7247265308601224
testing: with 20 nodes and 0.01 alpha, FDR at 3%: 0.5368715083798883
kfold-------
training: with 20 nodes and 0.02 alpha, FDR at 3%: 0.7225501286025174
testing: with 20 nodes and 0.02 alpha, FDR at 3%: 0.6921602765765005
testing: with 20 nodes and 0.02 alpha, FDR at 3%: 0.5262569832402234
kfold-------
training: with 20 nodes and 0.04 alpha, FDR at 3%: 0.6885687411572617
testing: with 20 nodes and 0.04 alpha, FDR at 3%: 0.6782780427131824
testing: with 20 nodes and 0.04 alpha, FDR at 3%: 0.5145251396648044
kfold-------
training: with 20 nodes and 0.06 alpha, FDR at 3%: 0.6825042559239172
testing: with 20 nodes and 0.06 alpha, FDR at 3%: 0.6597214866678064
testing: with 20 nodes and 0.06 alpha, FDR at 3%: 0.4888268156424581
kfold-------
training: with 20 nodes and 0.08 alpha, FDR at 3%: 0.6685910426814755
testing: with 20 nodes and 0.08 alpha, FDR at 

In [25]:
for n in range(1,21,1):
    for m in [0.001,0.002,0.004,0.006,0.008,0.01]:
        fdr_train = 0
        fdr_test = 0
        fdr_oot = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
            
            NN_oot_pred = NN.predict_proba(x_oot)[:,1]
            ind = np.argsort(NN_oot_pred)
            NN_y_oot_sorted = np.take_along_axis(y_oot, ind, axis=0)
            NN_oot_pred_sorted = np.take_along_axis(NN_oot_pred, ind, axis=0)
            NN_y_oot_sorted_3per = NN_y_oot_sorted[-int(len(NN_y_oot_sorted)*0.03):-1]
            NN_y_oot_sorted_3per_fraud = NN_y_oot_sorted_3per[NN_y_oot_sorted_3per==1]
            y_oot_fraud = len(y_oot[y_oot==1])
            fdr_oot += len(NN_y_oot_sorted_3per_fraud)/y_oot_fraud

        print(f'training: with {n} nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with {n} nodes and {m} alpha, FDR at 3%:', fdr_test/10)
        print(f'oot: with {n} nodes and {m} alpha, FDR at 3%:', fdr_oot/10)

kfold-------
training: with 1 nodes and 0.001 alpha, FDR at 3%: 0.5173710744858877
testing: with 1 nodes and 0.001 alpha, FDR at 3%: 0.5091150021972878
oot: with 1 nodes and 0.001 alpha, FDR at 3%: 0.3111731843575419
kfold-------
training: with 1 nodes and 0.002 alpha, FDR at 3%: 0.4594859468005142
testing: with 1 nodes and 0.002 alpha, FDR at 3%: 0.45230347674187243
oot: with 1 nodes and 0.002 alpha, FDR at 3%: 0.2731843575418994
kfold-------
training: with 1 nodes and 0.004 alpha, FDR at 3%: 0.45947555115288496
testing: with 1 nodes and 0.004 alpha, FDR at 3%: 0.45402318431064115
oot: with 1 nodes and 0.004 alpha, FDR at 3%: 0.27486033519553066
kfold-------
training: with 1 nodes and 0.006 alpha, FDR at 3%: 0.32846638229109393
testing: with 1 nodes and 0.006 alpha, FDR at 3%: 0.3208114326476699
oot: with 1 nodes and 0.006 alpha, FDR at 3%: 0.21452513966480447
kfold-------
training: with 1 nodes and 0.008 alpha, FDR at 3%: 0.6501506159968075
testing: with 1 nodes and 0.008 alpha, FDR 

training: with 7 nodes and 0.004 alpha, FDR at 3%: 0.7189961688326707
testing: with 7 nodes and 0.004 alpha, FDR at 3%: 0.6916048374265176
oot: with 7 nodes and 0.004 alpha, FDR at 3%: 0.4754189944134078
kfold-------
training: with 7 nodes and 0.006 alpha, FDR at 3%: 0.7323427466592614
testing: with 7 nodes and 0.006 alpha, FDR at 3%: 0.696669239574083
oot: with 7 nodes and 0.006 alpha, FDR at 3%: 0.49329608938547487
kfold-------
training: with 7 nodes and 0.008 alpha, FDR at 3%: 0.7299654476908466
testing: with 7 nodes and 0.008 alpha, FDR at 3%: 0.7024510516372183
oot: with 7 nodes and 0.008 alpha, FDR at 3%: 0.5251396648044693
kfold-------
training: with 7 nodes and 0.01 alpha, FDR at 3%: 0.7245463625439997
testing: with 7 nodes and 0.01 alpha, FDR at 3%: 0.6945707793137623
oot: with 7 nodes and 0.01 alpha, FDR at 3%: 0.5229050279329609
kfold-------
training: with 8 nodes and 0.001 alpha, FDR at 3%: 0.7394519555750523
testing: with 8 nodes and 0.001 alpha, FDR at 3%: 0.7047419346872

training: with 13 nodes and 0.008 alpha, FDR at 3%: 0.7570085927634924
testing: with 13 nodes and 0.008 alpha, FDR at 3%: 0.7252350609441703
oot: with 13 nodes and 0.008 alpha, FDR at 3%: 0.5195530726256983
kfold-------
training: with 13 nodes and 0.01 alpha, FDR at 3%: 0.7433205989433784
testing: with 13 nodes and 0.01 alpha, FDR at 3%: 0.7151055963601538
oot: with 13 nodes and 0.01 alpha, FDR at 3%: 0.5039106145251396
kfold-------
training: with 14 nodes and 0.001 alpha, FDR at 3%: 0.7726075656042312
testing: with 14 nodes and 0.001 alpha, FDR at 3%: 0.7387819157949101
oot: with 14 nodes and 0.001 alpha, FDR at 3%: 0.44581005586592176
kfold-------
training: with 14 nodes and 0.002 alpha, FDR at 3%: 0.7752796399678102
testing: with 14 nodes and 0.002 alpha, FDR at 3%: 0.7378311388381082
oot: with 14 nodes and 0.002 alpha, FDR at 3%: 0.488268156424581
kfold-------
training: with 14 nodes and 0.004 alpha, FDR at 3%: 0.7727800903423447
testing: with 14 nodes and 0.004 alpha, FDR at 3%: 0

training: with 20 nodes and 0.001 alpha, FDR at 3%: 0.7908002051160941
testing: with 20 nodes and 0.001 alpha, FDR at 3%: 0.7424921058324858
oot: with 20 nodes and 0.001 alpha, FDR at 3%: 0.4972067039106145
kfold-------
training: with 20 nodes and 0.002 alpha, FDR at 3%: 0.7865436862238562
testing: with 20 nodes and 0.002 alpha, FDR at 3%: 0.7439995431905453
oot: with 20 nodes and 0.002 alpha, FDR at 3%: 0.4837988826815643
kfold-------
training: with 20 nodes and 0.004 alpha, FDR at 3%: 0.7827561446779696
testing: with 20 nodes and 0.004 alpha, FDR at 3%: 0.7285165448633191
oot: with 20 nodes and 0.004 alpha, FDR at 3%: 0.4916201117318435
kfold-------
training: with 20 nodes and 0.006 alpha, FDR at 3%: 0.7766058169232363
testing: with 20 nodes and 0.006 alpha, FDR at 3%: 0.7428406793298186
oot: with 20 nodes and 0.006 alpha, FDR at 3%: 0.5128491620111733
kfold-------
training: with 20 nodes and 0.008 alpha, FDR at 3%: 0.763802382943111
testing: with 20 nodes and 0.008 alpha, FDR at 3%:

# Final model

In [65]:
import pandas as pd
import scipy.stats as sps
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [66]:
oot = pd.read_csv("oot_scaled_final.csv", index_col=0)
tt = pd.read_csv("train_test_final.csv", index_col=0)

In [67]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(tt, test_size=0.2, random_state=42, stratify=tt['fraud'])

In [68]:
select = ['card_ps_3d_total',
 'card-state_ps_3d_total',
 'card-state_ps_14d_total',
 'card-state_ps_3d_maximum',
 'card-state_ps_1d_total',
 'card-state_ps_7d_maximum',
 'card-state_ps_30d_total',
 'card_ps_0d_total',
 'card-state_ps_1d_maximum',
 'card-merchant_ps_3d_total',
 'card_ps_0d_maximum',
 'card-state_ps_3d_average',
 'card-state_ps_1d_average',
 'card-state_ps_1d_median',
 'card-zip_ps_30d_total',
 'card-state_ps_7d_average',
 'card-zip_ps_3d_maximum',
 'card-state_ps_3d_median',
 'card-merchant_ps_30d_total',
 'card-zip_ps_1d_total',
 'card-zip_ps_1d_maximum',
 'card-state_ps_14d_average',
 'card-merchant_ps_1d_total',
 'card_ps_30d_average',
 'card-state_ps_7d_median',
 'merch#_ps_3d_total',
 'card-state_ps_0d_maximum',
 'card-state_ps_14d_median',
 'card-state_ps_0d_total',
 'card-state_ps_0d_average']
len(select)

30

In [69]:
X_train = np.array(train[train.columns.intersection(select)])
Y_train = np.array(train['fraud'])
X_test = np.array(test[test.columns.intersection(select)])
Y_test = np.array(test['fraud'])
X_oot = np.array(oot[oot.columns.intersection(select)])
Y_oot = np.array(oot['fraud'])
X_tt = np.array(tt[tt.columns.intersection(select)])
Y_tt = np.array(tt['fraud'])

## build model on training set, predict on both training and testing sets

In [70]:
from sklearn.neural_network import MLPClassifier

In [71]:
NN = MLPClassifier(
    hidden_layer_sizes=(18,),
    activation='relu',
    learning_rate='adaptive',
    max_iter=10000,
    learning_rate_init=.01,
    alpha=0.001)

NN.fit(X_train, Y_train)

NN_train_pred = NN.predict_proba(X_train)
NN_test_pred = NN.predict_proba(X_test)

In [72]:
train1=train.copy()
train1['pred_label']=NN_train_pred[:,1]
train1.to_csv('final model_train pred.csv')

In [73]:
test1=test.copy()
test1['pred_label']=NN_test_pred[:,1]
test1.to_csv('final model_test pred.csv')

## build model on training+testing set, predict on oot set

In [74]:
NN2 = MLPClassifier(
    hidden_layer_sizes=(18,),
    activation='relu',
    learning_rate='adaptive',
    max_iter=10000,
    learning_rate_init=.01,
    alpha=0.001)

NN2.fit(X, Y)

NN2_oot_pred = NN2.predict_proba(X_oot)

In [75]:
oot1=oot.copy()
oot1['pred_label']=NN2_oot_pred[:,1]
oot1.to_csv('final model_oot pred.csv')

# without oot data

In [2]:
data = pd.read_csv("train_test_final.csv", index_col=0)

In [3]:
data

,card_ps_3d_total,card-state_ps_3d_total,card-state_ps_14d_total,card-state_ps_3d_maximum,card-state_ps_1d_total,card-state_ps_7d_maximum,card-state_ps_30d_total,card_ps_0d_total,card-state_ps_1d_maximum,card-merchant_ps_3d_total,...,card-state_ps_14d_average,card-merchant_ps_1d_total,card_ps_30d_average,card-state_ps_7d_median,merch#_ps_3d_total,card-state_ps_0d_maximum,card-state_ps_14d_median,card-state_ps_0d_total,card-state_ps_0d_average,fraud
3338,0.239431,0.000000,0.405804,0.000000,0.000000,1.057503,0.293871,0.000000,0.000000,0.000000,...,1.423042,0.000000,0.224369,1.608665,0.087707,0.000000,1.481152,0.000000,0.000000,0
3339,0.109461,0.052926,0.122207,0.015794,0.000000,0.016556,0.066435,0.000000,0.000000,0.050124,...,-0.002238,0.000000,-0.582122,0.007515,0.131778,0.000000,0.000110,0.000000,0.000000,0
3340,1.303699,0.207725,0.606770,0.452024,0.097819,0.399990,0.455039,0.000000,0.357360,0.096802,...,0.461099,0.099204,0.328960,0.551246,0.044923,0.000000,0.547433,0.000000,0.000000,0
3341,0.110170,0.054255,0.123333,0.015794,0.001374,0.016556,0.067337,0.001415,0.005603,0.051484,...,-0.002240,0.001394,-0.582219,0.007515,0.132670,0.007521,0.000110,0.002375,0.012225,0
3342,-0.025350,0.000000,-0.013172,0.000000,0.000000,0.000000,-0.042135,0.000000,0.000000,0.000000,...,-0.010154,0.000000,-0.602674,0.000000,0.004678,0.000000,-0.006842,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83965,-0.025350,0.000000,-0.013172,0.000000,0.000000,0.000000,-0.042135,0.000000,0.000000,0.000000,...,-0.010154,0.000000,0.126709,0.000000,0.105256,0.000000,-0.006842,0.000000,0.000000,0
83966,0.003864,0.054733,0.033181,0.218662,0.056607,0.193491,-0.004961,0.058284,0.230753,0.056019,...,0.306967,0.057409,-0.233441,0.355947,0.103055,0.309759,0.322405,0.097827,0.503497,0
83967,-0.022112,0.000000,-0.013172,0.000000,0.000000,0.000000,-0.042135,0.000000,0.000000,0.000000,...,-0.010154,0.000000,-0.157255,0.000000,-0.018568,0.000000,-0.006842,0.000000,0.000000,0
83968,-0.025350,0.000000,-0.013172,0.000000,0.000000,0.000000,-0.042135,0.000000,0.000000,0.000000,...,-0.010154,0.000000,-0.602674,0.000000,-0.018568,0.000000,-0.006842,0.000000,0.000000,0


In [4]:
data.dtypes

card_ps_3d_total              float64
card-state_ps_3d_total        float64
card-state_ps_14d_total       float64
card-state_ps_3d_maximum      float64
card-state_ps_1d_total        float64
card-state_ps_7d_maximum      float64
card-state_ps_30d_total       float64
card_ps_0d_total              float64
card-state_ps_1d_maximum      float64
card-merchant_ps_3d_total     float64
card_ps_0d_maximum            float64
card-state_ps_3d_average      float64
card-state_ps_1d_average      float64
card-state_ps_1d_median       float64
card-zip_ps_30d_total         float64
card-state_ps_7d_average      float64
card-zip_ps_3d_maximum        float64
card-state_ps_3d_median       float64
card-merchant_ps_30d_total    float64
card-zip_ps_1d_total          float64
card-zip_ps_1d_maximum        float64
card-state_ps_14d_average     float64
card-merchant_ps_1d_total     float64
card_ps_30d_average           float64
card-state_ps_7d_median       float64
merch#_ps_3d_total            float64
card-state_p

In [5]:
from sklearn.model_selection import KFold

In [6]:
from sklearn.neural_network import MLPClassifier

In [7]:
X=data.iloc[:,:30].values
y=data.iloc[:,30].values

In [8]:
X

array([[ 2.39430514e-01,  0.00000000e+00,  4.05804364e-01, ...,
         1.48115173e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.09460512e-01,  5.29263073e-02,  1.22207082e-01, ...,
         1.09761016e-04,  0.00000000e+00,  0.00000000e+00],
       [ 1.30369869e+00,  2.07724504e-01,  6.06770441e-01, ...,
         5.47433065e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-2.21115541e-02,  0.00000000e+00, -1.31722136e-02, ...,
        -6.84176995e-03,  0.00000000e+00,  0.00000000e+00],
       [-2.53497667e-02,  0.00000000e+00, -1.31722136e-02, ...,
        -6.84176995e-03,  0.00000000e+00,  0.00000000e+00],
       [-2.53497667e-02,  0.00000000e+00, -1.31722136e-02, ...,
        -6.84176995e-03,  0.00000000e+00,  0.00000000e+00]])

In [9]:
for n in range(1,6,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with {n} nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with {n} nodes and {m} alpha, FDR at 3%:', fdr_test/10)

kfold-------
training: with 1 nodes and 0.01 alpha, FDR at 3%: 0.6501515923323347
testing: with 1 nodes and 0.01 alpha, FDR at 3%: 0.6339781718871811
kfold-------
training: with 1 nodes and 0.02 alpha, FDR at 3%: 0.46003220309957654
testing: with 1 nodes and 0.02 alpha, FDR at 3%: 0.44641570132658687
kfold-------
training: with 1 nodes and 0.04 alpha, FDR at 3%: 0.5194203301490687
testing: with 1 nodes and 0.04 alpha, FDR at 3%: 0.5118869038037426
kfold-------
training: with 1 nodes and 0.06 alpha, FDR at 3%: 0.4572206827565356
testing: with 1 nodes and 0.06 alpha, FDR at 3%: 0.45764696132173033
kfold-------
training: with 1 nodes and 0.08 alpha, FDR at 3%: 0.3949817523121671
testing: with 1 nodes and 0.08 alpha, FDR at 3%: 0.3824470173395838
kfold-------
training: with 1 nodes and 0.1 alpha, FDR at 3%: 0.5846871839633498
testing: with 1 nodes and 0.1 alpha, FDR at 3%: 0.580840496041994
kfold-------
training: with 2 nodes and 0.01 alpha, FDR at 3%: 0.5879685275465809
testing: with 2 no

In [ ]:
for n in range(5,14,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with {n} nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with {n} nodes and {m} alpha, FDR at 3%:', fdr_test/10)

kfold-------
training: with 5 nodes and 0.01 alpha, FDR at 3%: 0.6748412746575119
testing: with 5 nodes and 0.01 alpha, FDR at 3%: 0.653090778570588
kfold-------
training: with 5 nodes and 0.02 alpha, FDR at 3%: 0.6765113557314372
testing: with 5 nodes and 0.02 alpha, FDR at 3%: 0.6534407025922411
kfold-------
training: with 5 nodes and 0.04 alpha, FDR at 3%: 0.6696747365639325
testing: with 5 nodes and 0.04 alpha, FDR at 3%: 0.6564130331590626
kfold-------
training: with 5 nodes and 0.06 alpha, FDR at 3%: 0.6605613633100769
testing: with 5 nodes and 0.06 alpha, FDR at 3%: 0.6347167339808466
kfold-------
training: with 5 nodes and 0.08 alpha, FDR at 3%: 0.6690755738396718
testing: with 5 nodes and 0.08 alpha, FDR at 3%: 0.6433705484351101
kfold-------
training: with 5 nodes and 0.1 alpha, FDR at 3%: 0.6619272790461717
testing: with 5 nodes and 0.1 alpha, FDR at 3%: 0.6468653591028926
kfold-------
training: with 6 nodes and 0.01 alpha, FDR at 3%: 0.6940610752764533
testing: with 6 nodes

In [9]:
for n in range(13,21,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with {n} nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with {n} nodes and {m} alpha, FDR at 3%:', fdr_test/10)

kfold-------
training: with 13 nodes and 0.01 alpha, FDR at 3%: 0.7517755944520197
testing: with 13 nodes and 0.01 alpha, FDR at 3%: 0.7089178487379398
kfold-------
training: with 13 nodes and 0.02 alpha, FDR at 3%: 0.7108639312633025
testing: with 13 nodes and 0.02 alpha, FDR at 3%: 0.6861470571690276
kfold-------
training: with 13 nodes and 0.04 alpha, FDR at 3%: 0.6833164910931402
testing: with 13 nodes and 0.04 alpha, FDR at 3%: 0.6598332843107935
kfold-------
training: with 13 nodes and 0.06 alpha, FDR at 3%: 0.6740506625305822
testing: with 13 nodes and 0.06 alpha, FDR at 3%: 0.6596864216050633
kfold-------
training: with 13 nodes and 0.08 alpha, FDR at 3%: 0.6682956036407088
testing: with 13 nodes and 0.08 alpha, FDR at 3%: 0.6481716571582259
kfold-------
training: with 13 nodes and 0.1 alpha, FDR at 3%: 0.6675510877609662
testing: with 13 nodes and 0.1 alpha, FDR at 3%: 0.6421557281272234
kfold-------
training: with 14 nodes and 0.01 alpha, FDR at 3%: 0.7403175951384527
testing

In [10]:
for n in range(5,21,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,5),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with 1st {n} nodes, 2nd 5 nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with 1st {n} nodes, 2nd 5 nodes and {m} alpha, FDR at 3%:', fdr_test/10)

kfold-------
training: with 1st 5 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7198439841349897
testing: with 1st 5 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.6775878417860248
kfold-------
training: with 1st 5 nodes, 2nd 5 nodes and 0.02 alpha, FDR at 3%: 0.6792570969755628
testing: with 1st 5 nodes, 2nd 5 nodes and 0.02 alpha, FDR at 3%: 0.6574894687039683
kfold-------
training: with 1st 5 nodes, 2nd 5 nodes and 0.04 alpha, FDR at 3%: 0.6953175683634445
testing: with 1st 5 nodes, 2nd 5 nodes and 0.04 alpha, FDR at 3%: 0.6747804001871016
kfold-------
training: with 1st 5 nodes, 2nd 5 nodes and 0.06 alpha, FDR at 3%: 0.682097144090301
testing: with 1st 5 nodes, 2nd 5 nodes and 0.06 alpha, FDR at 3%: 0.6526623601386795
kfold-------
training: with 1st 5 nodes, 2nd 5 nodes and 0.08 alpha, FDR at 3%: 0.667128186976812
testing: with 1st 5 nodes, 2nd 5 nodes and 0.08 alpha, FDR at 3%: 0.6570633747539001
kfold-------
training: with 1st 5 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6

training: with 1st 12 nodes, 2nd 5 nodes and 0.06 alpha, FDR at 3%: 0.6998659089991569
testing: with 1st 12 nodes, 2nd 5 nodes and 0.06 alpha, FDR at 3%: 0.664856741904781
kfold-------
training: with 1st 12 nodes, 2nd 5 nodes and 0.08 alpha, FDR at 3%: 0.6850007357705031
testing: with 1st 12 nodes, 2nd 5 nodes and 0.08 alpha, FDR at 3%: 0.649749317416393
kfold-------
training: with 1st 12 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6812666436266239
testing: with 1st 12 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6550126215351091
kfold-------
training: with 1st 13 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7724713914249838
testing: with 1st 13 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.725554287884933
kfold-------
training: with 1st 13 nodes, 2nd 5 nodes and 0.02 alpha, FDR at 3%: 0.7498645788165336
testing: with 1st 13 nodes, 2nd 5 nodes and 0.02 alpha, FDR at 3%: 0.7267068937232178
kfold-------
training: with 1st 13 nodes, 2nd 5 nodes and 0.04 alpha, FDR at 3%: 0.73221

training: with 1st 20 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7846900762734423
testing: with 1st 20 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7439989528217208
kfold-------
training: with 1st 20 nodes, 2nd 5 nodes and 0.02 alpha, FDR at 3%: 0.7615087050264834
testing: with 1st 20 nodes, 2nd 5 nodes and 0.02 alpha, FDR at 3%: 0.7260322453189052
kfold-------
training: with 1st 20 nodes, 2nd 5 nodes and 0.04 alpha, FDR at 3%: 0.7341704385603169
testing: with 1st 20 nodes, 2nd 5 nodes and 0.04 alpha, FDR at 3%: 0.7080377133252795
kfold-------
training: with 1st 20 nodes, 2nd 5 nodes and 0.06 alpha, FDR at 3%: 0.7181927410536891
testing: with 1st 20 nodes, 2nd 5 nodes and 0.06 alpha, FDR at 3%: 0.6896605153254323
kfold-------
training: with 1st 20 nodes, 2nd 5 nodes and 0.08 alpha, FDR at 3%: 0.6934836995595572
testing: with 1st 20 nodes, 2nd 5 nodes and 0.08 alpha, FDR at 3%: 0.6699670782493817
kfold-------
training: with 1st 20 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6

In [ ]:
for n in range(5,21,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,10),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with 1st {n} nodes, 2nd 10 nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with 1st {n} nodes, 2nd 10 nodes and {m} alpha, FDR at 3%:', fdr_test/10)

kfold-------
training: with 1st 5 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.697748308122377
testing: with 1st 5 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.6725008006944718
kfold-------
training: with 1st 5 nodes, 2nd 10 nodes and 0.02 alpha, FDR at 3%: 0.7228748692697232
testing: with 1st 5 nodes, 2nd 10 nodes and 0.02 alpha, FDR at 3%: 0.7059753288207088
kfold-------
training: with 1st 5 nodes, 2nd 10 nodes and 0.04 alpha, FDR at 3%: 0.6871297800445284
testing: with 1st 5 nodes, 2nd 10 nodes and 0.04 alpha, FDR at 3%: 0.6606449204798241
kfold-------
training: with 1st 5 nodes, 2nd 10 nodes and 0.06 alpha, FDR at 3%: 0.6827399341857514
testing: with 1st 5 nodes, 2nd 10 nodes and 0.06 alpha, FDR at 3%: 0.6554153597019461
kfold-------
training: with 1st 5 nodes, 2nd 10 nodes and 0.08 alpha, FDR at 3%: 0.6806291409200722
testing: with 1st 5 nodes, 2nd 10 nodes and 0.08 alpha, FDR at 3%: 0.6398779322548893
kfold-------
training: with 1st 5 nodes, 2nd 10 nodes and 0.1 alpha, FD

training: with 1st 12 nodes, 2nd 10 nodes and 0.06 alpha, FDR at 3%: 0.7052812747242777
testing: with 1st 12 nodes, 2nd 10 nodes and 0.06 alpha, FDR at 3%: 0.6813988540703357
kfold-------
training: with 1st 12 nodes, 2nd 10 nodes and 0.08 alpha, FDR at 3%: 0.6879665193741848
testing: with 1st 12 nodes, 2nd 10 nodes and 0.08 alpha, FDR at 3%: 0.674705311551002
kfold-------
training: with 1st 12 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6860717850477754
testing: with 1st 12 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6625997315298109
kfold-------
training: with 1st 13 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7706383248080761
testing: with 1st 13 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7368021013978352
kfold-------
training: with 1st 13 nodes, 2nd 10 nodes and 0.02 alpha, FDR at 3%: 0.7591648850711363
testing: with 1st 13 nodes, 2nd 10 nodes and 0.02 alpha, FDR at 3%: 0.7414730458259076
kfold-------
training: with 1st 13 nodes, 2nd 10 nodes and 0.04 alpha, FDR a

In [ ]:
for n in range(5,21,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,15),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with 1st {n} nodes, 2nd 15 nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with 1st {n} nodes, 2nd 15 nodes and {m} alpha, FDR at 3%:', fdr_test/10)

In [ ]:
for n in range(5,21,1):
    for m in [0.01,0.02,0.04,0.06,0.08,0.1]:
        fdr_train = 0
        fdr_test = 0

        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        print("kfold-------")
        for train_index, test_index in kf.split(X):
            NN = MLPClassifier(
                hidden_layer_sizes=(n,20),
                activation='relu',
                learning_rate='adaptive',
                max_iter=10000,
                learning_rate_init=.01,
                alpha=m)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            NN.fit(X_train, y_train)

            NN_train_pred = NN.predict_proba(X_train)[:,1]
            ind = np.argsort(NN_train_pred)
            NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
            NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
            NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
            NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
            y_train_fraud = len(y_train[y_train==1])
            fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

            NN_test_pred = NN.predict_proba(X_test)[:,1]
            ind = np.argsort(NN_test_pred)
            NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
            NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
            NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
            NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
            y_test_fraud = len(y_test[y_test==1])
            fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        print(f'training: with 1st {n} nodes, 2nd 20 nodes and {m} alpha, FDR at 3%:', fdr_train/10)
        print(f'testing: with 1st {n} nodes, 2nd 20 nodes and {m} alpha, FDR at 3%:', fdr_test/10)

# 后面的有问题不看

In [7]:
# hidden_layer_size range(5,21,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.01)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.01 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.01 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.01 alpha, FDR at 3%: 0.6942857142857143
In testing set, with 5 nodes and 0.01 alpha, FDR at 3%: 0.710204081632653
In training set, with 6 nodes and 0.01 alpha, FDR at 3%: 0.7150649350649351
In testing set, with 6 nodes and 0.01 alpha, FDR at 3%: 0.7163265306122448
In training set, with 7 nodes and 0.01 alpha, FDR at 3%: 0.7023376623376623
In testing set, with 7 nodes and 0.01 alpha, FDR at 3%: 0.7244897959183673
In training set, with 8 nodes and 0.01 alpha, FDR at 3%: 0.7038961038961039
In testing set, with 8 nodes and 0.01 alpha, FDR at 3%: 0.7061224489795918
In training set, with 9 nodes and 0.01 alpha, FDR at 3%: 0.6966233766233766
In testing set, with 9 nodes and 0.01 alpha, FDR at 3%: 0.6959183673469388
In training set, with 10 nodes and 0.01 alpha, FDR at 3%: 0.7218181818181819
In testing set, with 10 nodes and 0.01 alpha, FDR at 3%: 0.7326530612244898
In training set, with 11 nodes and 0.01 alpha, FDR at 3%: 0.7277922077922079
In testing set, 

In [8]:
# hidden_layer_size range(5,21,1) & alpha = 0.05

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.05)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud
    
        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.05 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.05 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.05 alpha, FDR at 3%: 0.6625974025974026
In testing set, with 5 nodes and 0.05 alpha, FDR at 3%: 0.6755102040816326
In training set, with 6 nodes and 0.05 alpha, FDR at 3%: 0.6618181818181819
In testing set, with 6 nodes and 0.05 alpha, FDR at 3%: 0.6755102040816326
In training set, with 7 nodes and 0.05 alpha, FDR at 3%: 0.6688311688311688
In testing set, with 7 nodes and 0.05 alpha, FDR at 3%: 0.6795918367346939
In training set, with 8 nodes and 0.05 alpha, FDR at 3%: 0.6667532467532468
In testing set, with 8 nodes and 0.05 alpha, FDR at 3%: 0.6816326530612244
In training set, with 9 nodes and 0.05 alpha, FDR at 3%: 0.6735064935064935
In testing set, with 9 nodes and 0.05 alpha, FDR at 3%: 0.7061224489795919
In training set, with 10 nodes and 0.05 alpha, FDR at 3%: 0.6701298701298701
In testing set, with 10 nodes and 0.05 alpha, FDR at 3%: 0.6775510204081633
In training set, with 11 nodes and 0.05 alpha, FDR at 3%: 0.6797402597402598
In testing set,

In [9]:
# hidden_layer_size range(5,21,1) & alpha = 0.1

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.1)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud
    
        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.1 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.1 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.1 alpha, FDR at 3%: 0.6592207792207792
In testing set, with 5 nodes and 0.1 alpha, FDR at 3%: 0.6653061224489796
In training set, with 6 nodes and 0.1 alpha, FDR at 3%: 0.6610389610389611
In testing set, with 6 nodes and 0.1 alpha, FDR at 3%: 0.6714285714285715
In training set, with 7 nodes and 0.1 alpha, FDR at 3%: 0.6550649350649351
In testing set, with 7 nodes and 0.1 alpha, FDR at 3%: 0.6714285714285714
In training set, with 8 nodes and 0.1 alpha, FDR at 3%: 0.6566233766233767
In testing set, with 8 nodes and 0.1 alpha, FDR at 3%: 0.6693877551020408
In training set, with 9 nodes and 0.1 alpha, FDR at 3%: 0.6592207792207792
In testing set, with 9 nodes and 0.1 alpha, FDR at 3%: 0.6755102040816326
In training set, with 10 nodes and 0.1 alpha, FDR at 3%: 0.6587012987012987
In testing set, with 10 nodes and 0.1 alpha, FDR at 3%: 0.6775510204081632
In training set, with 11 nodes and 0.1 alpha, FDR at 3%: 0.6584415584415584
In testing set, with 11 node

In [7]:
# hidden_layer_size range(5,21,1) & alpha = 0.02

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.02)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud
    
        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.02 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.02 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.02 alpha, FDR at 3%: 0.6799999999999999
In testing set, with 5 nodes and 0.02 alpha, FDR at 3%: 0.689795918367347
In training set, with 6 nodes and 0.02 alpha, FDR at 3%: 0.6610389610389611
In testing set, with 6 nodes and 0.02 alpha, FDR at 3%: 0.6632653061224489
In training set, with 7 nodes and 0.02 alpha, FDR at 3%: 0.692987012987013
In testing set, with 7 nodes and 0.02 alpha, FDR at 3%: 0.6877551020408164
In training set, with 8 nodes and 0.02 alpha, FDR at 3%: 0.6903896103896103
In testing set, with 8 nodes and 0.02 alpha, FDR at 3%: 0.7
In training set, with 9 nodes and 0.02 alpha, FDR at 3%: 0.6953246753246753
In testing set, with 9 nodes and 0.02 alpha, FDR at 3%: 0.7040816326530612
In training set, with 10 nodes and 0.02 alpha, FDR at 3%: 0.7057142857142857
In testing set, with 10 nodes and 0.02 alpha, FDR at 3%: 0.7183673469387755
In training set, with 11 nodes and 0.02 alpha, FDR at 3%: 0.6903896103896103
In testing set, with 11 nodes an

In [8]:
# hidden_layer_size range(5,21,1) & alpha = 0.04

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.04)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud
    
        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.04 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.04 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.04 alpha, FDR at 3%: 0.6672727272727274
In testing set, with 5 nodes and 0.04 alpha, FDR at 3%: 0.673469387755102
In training set, with 6 nodes and 0.04 alpha, FDR at 3%: 0.67012987012987
In testing set, with 6 nodes and 0.04 alpha, FDR at 3%: 0.6979591836734694
In training set, with 7 nodes and 0.04 alpha, FDR at 3%: 0.677142857142857
In testing set, with 7 nodes and 0.04 alpha, FDR at 3%: 0.7
In training set, with 8 nodes and 0.04 alpha, FDR at 3%: 0.6748051948051947
In testing set, with 8 nodes and 0.04 alpha, FDR at 3%: 0.6857142857142857
In training set, with 9 nodes and 0.04 alpha, FDR at 3%: 0.6729870129870129
In testing set, with 9 nodes and 0.04 alpha, FDR at 3%: 0.6979591836734694
In training set, with 10 nodes and 0.04 alpha, FDR at 3%: 0.6787012987012987
In testing set, with 10 nodes and 0.04 alpha, FDR at 3%: 0.7020408163265305
In training set, with 11 nodes and 0.04 alpha, FDR at 3%: 0.6779220779220779
In testing set, with 11 nodes and 

In [7]:
# hidden_layer_size range(5,21,1) & alpha = 0.06

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.06)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud
    
        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.06 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.06 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.06 alpha, FDR at 3%: 0.6633766233766233
In testing set, with 5 nodes and 0.06 alpha, FDR at 3%: 0.689795918367347
In training set, with 6 nodes and 0.06 alpha, FDR at 3%: 0.6615584415584416
In testing set, with 6 nodes and 0.06 alpha, FDR at 3%: 0.6714285714285714
In training set, with 7 nodes and 0.06 alpha, FDR at 3%: 0.6766233766233767
In testing set, with 7 nodes and 0.06 alpha, FDR at 3%: 0.6857142857142857
In training set, with 8 nodes and 0.06 alpha, FDR at 3%: 0.6688311688311688
In testing set, with 8 nodes and 0.06 alpha, FDR at 3%: 0.6734693877551019
In training set, with 9 nodes and 0.06 alpha, FDR at 3%: 0.6727272727272726
In testing set, with 9 nodes and 0.06 alpha, FDR at 3%: 0.6795918367346938
In training set, with 10 nodes and 0.06 alpha, FDR at 3%: 0.6719480519480518
In testing set, with 10 nodes and 0.06 alpha, FDR at 3%: 0.6857142857142857
In training set, with 11 nodes and 0.06 alpha, FDR at 3%: 0.6763636363636364
In testing set, 

In [7]:
# hidden_layer_size range(5,21,1) & alpha = 0.08

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0

    for i in range(5):
        
        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()
        
        NN = MLPClassifier(
            hidden_layer_sizes=(n,),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.08)
        
        NN.fit(x_train,y_train)
        
        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud
    
        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud
        
        i += 1
        
    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with {n} nodes and 0.08 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with {n} nodes and 0.08 alpha, FDR at 3%:', avg_fdr_test)
    
    n += 1

In training set, with 5 nodes and 0.08 alpha, FDR at 3%: 0.6605194805194805
In testing set, with 5 nodes and 0.08 alpha, FDR at 3%: 0.6714285714285715
In training set, with 6 nodes and 0.08 alpha, FDR at 3%: 0.6597402597402597
In testing set, with 6 nodes and 0.08 alpha, FDR at 3%: 0.6632653061224489
In training set, with 7 nodes and 0.08 alpha, FDR at 3%: 0.6662337662337662
In testing set, with 7 nodes and 0.08 alpha, FDR at 3%: 0.6714285714285715
In training set, with 8 nodes and 0.08 alpha, FDR at 3%: 0.6685714285714285
In testing set, with 8 nodes and 0.08 alpha, FDR at 3%: 0.6836734693877551
In training set, with 9 nodes and 0.08 alpha, FDR at 3%: 0.6690909090909091
In testing set, with 9 nodes and 0.08 alpha, FDR at 3%: 0.6877551020408164
In training set, with 10 nodes and 0.08 alpha, FDR at 3%: 0.6646753246753246
In testing set, with 10 nodes and 0.08 alpha, FDR at 3%: 0.6877551020408162
In training set, with 11 nodes and 0.08 alpha, FDR at 3%: 0.6690909090909091
In testing set,

In [8]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,5),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.01)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.6854545454545454
In testing set, with 1st 5 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.6979591836734695
In training set, with 1st 6 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7420779220779221
In testing set, with 1st 6 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7428571428571429
In training set, with 1st 7 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7402597402597403
In testing set, with 1st 7 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7408163265306122
In training set, with 1st 8 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.750909090909091
In testing set, with 1st 8 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.753061224489796
In training set, with 1st 9 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7584415584415585
In testing set, with 1st 9 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.7510204081632653
In training set, with 1st 10 nodes, 2nd 5 nodes and 0.01 alpha, FDR at 3%: 0.

In [9]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,10),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.01)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.6922077922077922
In testing set, with 1st 5 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.6979591836734695
In training set, with 1st 6 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7431168831168831
In testing set, with 1st 6 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.753061224489796
In training set, with 1st 7 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7431168831168831
In testing set, with 1st 7 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7285714285714285
In training set, with 1st 8 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7581818181818182
In testing set, with 1st 8 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7489795918367348
In training set, with 1st 9 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7457142857142858
In testing set, with 1st 9 nodes, 2nd 10 nodes and 0.01 alpha, FDR at 3%: 0.7510204081632652
In training set, with 1st 10 nodes, 2nd 10 nodes and 0.01 alpha, F

In [10]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,15),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.01)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7083116883116883
In testing set, with 1st 5 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7102040816326531
In training set, with 1st 6 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.732987012987013
In testing set, with 1st 6 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.746938775510204
In training set, with 1st 7 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7345454545454546
In testing set, with 1st 7 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7428571428571428
In training set, with 1st 8 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7503896103896104
In testing set, with 1st 8 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7693877551020408
In training set, with 1st 9 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7514285714285714
In testing set, with 1st 9 nodes, 2nd 15 nodes and 0.01 alpha, FDR at 3%: 0.7489795918367347
In training set, with 1st 10 nodes, 2nd 15 nodes and 0.01 alpha, FD

In [ ]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,20),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.01)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7363636363636363
In testing set, with 1st 5 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7285714285714285
In training set, with 1st 6 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7150649350649351
In testing set, with 1st 6 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7428571428571429
In training set, with 1st 7 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7519480519480519
In testing set, with 1st 7 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7489795918367348
In training set, with 1st 8 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7592207792207792
In testing set, with 1st 8 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7653061224489794
In training set, with 1st 9 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7548051948051947
In testing set, with 1st 9 nodes, 2nd 20 nodes and 0.01 alpha, FDR at 3%: 0.7571428571428572
In training set, with 1st 10 nodes, 2nd 20 nodes and 0.01 alpha, 

In [7]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,5),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.1)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6483116883116884
In testing set, with 1st 5 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6653061224489796
In training set, with 1st 6 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6714285714285714
In testing set, with 1st 6 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6979591836734694
In training set, with 1st 7 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6664935064935066
In testing set, with 1st 7 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.673469387755102
In training set, with 1st 8 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6745454545454546
In testing set, with 1st 8 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.7163265306122449
In training set, with 1st 9 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6766233766233766
In testing set, with 1st 9 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6979591836734694
In training set, with 1st 10 nodes, 2nd 5 nodes and 0.1 alpha, FDR at 3%: 0.6880519480

In [8]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,10),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.1)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6724675324675324
In testing set, with 1st 5 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6938775510204083
In training set, with 1st 6 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.655064935064935
In testing set, with 1st 6 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.673469387755102
In training set, with 1st 7 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6532467532467533
In testing set, with 1st 7 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6632653061224489
In training set, with 1st 8 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6607792207792207
In testing set, with 1st 8 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6755102040816326
In training set, with 1st 9 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.6696103896103895
In testing set, with 1st 9 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.7020408163265306
In training set, with 1st 10 nodes, 2nd 10 nodes and 0.1 alpha, FDR at 3%: 0.

In [9]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,15),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.1)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6483116883116883
In testing set, with 1st 5 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6653061224489796
In training set, with 1st 6 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6584415584415585
In testing set, with 1st 6 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6877551020408162
In training set, with 1st 7 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6690909090909091
In testing set, with 1st 7 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6979591836734694
In training set, with 1st 8 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6664935064935065
In testing set, with 1st 8 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.7081632653061224
In training set, with 1st 9 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6846753246753245
In testing set, with 1st 9 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 0.6938775510204083
In training set, with 1st 10 nodes, 2nd 15 nodes and 0.1 alpha, FDR at 3%: 

In [10]:
# 2 layers & hidden_layer_size range(5,11,1) & alpha = 0.01

for n in range(5,21,1):
    fdr_train = 0
    fdr_test = 0
        
    for i in range(5):

        kfolds = KFold(n_splits=10, shuffle=True, random_state=1)
        for train_index, test_index in kfolds.split(data):
            y_train = data.iloc[train_index, -1].to_numpy()
            y_test = data.iloc[test_index, -1].to_numpy()
            x_train = data.iloc[train_index, 0:-1].to_numpy()
            x_test = data.iloc[test_index, 0:-1].to_numpy()

        NN = MLPClassifier(
            hidden_layer_sizes=(n,20),
            activation='relu',
            learning_rate='adaptive',
            max_iter=10000,
            learning_rate_init=.01,
            alpha=.1)

        NN.fit(x_train,y_train)

        NN_train_pred = NN.predict_proba(x_train)[:,1]
        ind = np.argsort(NN_train_pred)
        NN_y_train_sorted = np.take_along_axis(y_train, ind, axis=0)
        NN_train_pred_sorted = np.take_along_axis(NN_train_pred, ind, axis=0)
        NN_y_train_sorted_3per = NN_y_train_sorted[-int(len(NN_y_train_sorted)*0.03):-1]
        NN_y_train_sorted_3per_fraud = NN_y_train_sorted_3per[NN_y_train_sorted_3per==1]
        y_train_fraud = len(y_train[y_train==1])
        fdr_train += len(NN_y_train_sorted_3per_fraud)/y_train_fraud

        NN_test_pred = NN.predict_proba(x_test)[:,1]
        ind = np.argsort(NN_test_pred)
        NN_y_test_sorted = np.take_along_axis(y_test, ind, axis=0)
        NN_test_pred_sorted = np.take_along_axis(NN_test_pred, ind, axis=0)
        NN_y_test_sorted_3per = NN_y_test_sorted[-int(len(NN_y_test_sorted)*0.03):-1]
        NN_y_test_sorted_3per_fraud = NN_y_test_sorted_3per[NN_y_test_sorted_3per==1]
        y_test_fraud = len(y_test[y_test==1])
        fdr_test += len(NN_y_test_sorted_3per_fraud)/y_test_fraud

        i += 1

    avg_fdr_train = fdr_train/5
    avg_fdr_test = fdr_test/5

    print(f'In training set, with 1st {n} nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_train)
    print(f'In testing set, with 1st {n} nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%:', avg_fdr_test)

    n += 1

In training set, with 1st 5 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6561038961038961
In testing set, with 1st 5 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6795918367346939
In training set, with 1st 6 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6581818181818182
In testing set, with 1st 6 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6693877551020407
In training set, with 1st 7 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6683116883116884
In testing set, with 1st 7 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.7020408163265306
In training set, with 1st 8 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6851948051948051
In testing set, with 1st 8 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.7183673469387756
In training set, with 1st 9 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.6844155844155844
In testing set, with 1st 9 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0.726530612244898
In training set, with 1st 10 nodes, 2nd 20 nodes and 0.1 alpha, FDR at 3%: 0